In [1]:
!pip install pyuvdata astropy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 4.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached astropy-6.1.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.0 MB)
  Using cached pyerfa-2.0.1.5-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (738 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Try to import required packages
try:
    from pyuvdata import UVData
    PYUVDATA_AVAILABLE = True
except ImportError:
    print("⚠️  pyuvdata not installed. Install with: pip install pyuvdata")
    PYUVDATA_AVAILABLE = False

try:
    from astropy.io import fits
    ASTROPY_AVAILABLE = True
except ImportError:
    print("⚠️  astropy not installed. Install with: pip install astropy")
    ASTROPY_AVAILABLE = False

# Configure matplotlib for notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Imports successful!")
print(f"   pyuvdata available: {PYUVDATA_AVAILABLE}")
print(f"   astropy available: {ASTROPY_AVAILABLE}")

# =============================================================================
# SECTION 1: Load UVFITS File
# =============================================================================

# 📁 Set your UVFITS file path here
UVFITS_FILE = "your_file.uvfits"  # Change this to your file path

# Optional: Use file browser widget
try:
    from ipywidgets import FileUpload, VBox, HTML
    from IPython.display import display
    
    print("\n📂 File Browser Available - Upload your UVFITS file below:")
    uploader = FileUpload(accept='.uvfits,.fits', multiple=False)
    display(uploader)
except ImportError:
    print("\n💡 Tip: Install ipywidgets for file browser: pip install ipywidgets")

def load_uvfits(filename):
    """Load UVFITS file using pyuvdata"""
    if not PYUVDATA_AVAILABLE:
        print("❌ pyuvdata is required but not installed")
        return None
    
    if not Path(filename).exists():
        print(f"❌ File not found: {filename}")
        return None
    
    print(f"📖 Loading: {filename}")
    uv = UVData()
    uv.read(filename)
    print("✅ File loaded successfully!")
    return uv

# Load the file (uncomment and modify the path)
# uv = load_uvfits(UVFITS_FILE)

# Or load from uploaded file
# if uploader.value:
#     # Get uploaded file
#     uploaded_file = list(uploader.value.values())[0]
#     filename = uploaded_file['metadata']['name']
#     
#     # Save temporarily
#     with open(filename, 'wb') as f:
#         f.write(uploaded_file['content'])
#     
#     uv = load_uvfits(filename)

# =============================================================================
# SECTION 2: Display Metadata and Summary
# =============================================================================

def display_metadata(uv):
    """Display comprehensive metadata from UVFITS file"""
    
    print("="*70)
    print("                    UVFITS FILE SUMMARY")
    print("="*70)
    
    print("\n📡 TELESCOPE & OBSERVATION")
    print("-" * 70)
    print(f"  Telescope:        {uv.telescope_name}")
    print(f"  Instrument:       {uv.instrument}")
    print(f"  Object:           {uv.object_name if hasattr(uv, 'object_name') else 'N/A'}")
    
    print("\n⏱️  TIME INFORMATION")
    print("-" * 70)
    print(f"  Number of times:  {uv.Ntimes}")
    print(f"  Integration time: {uv.integration_time[0]:.3f} seconds")
    print(f"  Start time (JD):  {uv.time_array.min():.6f}")
    print(f"  End time (JD):    {uv.time_array.max():.6f}")
    duration = (uv.time_array.max() - uv.time_array.min()) * 24 * 3600
    print(f"  Duration:         {duration:.1f} seconds ({duration/60:.1f} minutes)")
    
    print("\n📻 FREQUENCY INFORMATION")
    print("-" * 70)
    print(f"  Number of freqs:  {uv.Nfreqs}")
    print(f"  Frequency range:  {uv.freq_array.min()/1e6:.3f} - {uv.freq_array.max()/1e6:.3f} MHz")
    print(f"  Channel width:    {uv.channel_width/1e3:.3f} kHz")
    print(f"  Bandwidth:        {(uv.freq_array.max() - uv.freq_array.min())/1e6:.3f} MHz")
    
    print("\n📶 POLARIZATION INFORMATION")
    print("-" * 70)
    print(f"  Number of pols:   {uv.Npols}")
    print(f"  Polarizations:    {uv.polarization_array}")
    pol_names = {-5: 'XX', -6: 'YY', -7: 'XY', -8: 'YX', 
                 1: 'I', 2: 'Q', 3: 'U', 4: 'V'}
    pol_str = [pol_names.get(p, str(p)) for p in uv.polarization_array]
    print(f"  Pol names:        {pol_str}")
    
    print("\n🔭 BASELINE & ANTENNA INFORMATION")
    print("-" * 70)
    print(f"  Number of baselines: {uv.Nbls}")
    print(f"  Number of antennas:  {uv.Nants_data}")
    print(f"  Antenna numbers:     {sorted(np.unique(uv.ant_1_array))}")
    
    print("\n🎯 PHASE CENTER")
    print("-" * 70)
    print(f"  RA:  {np.degrees(uv.phase_center_ra):.6f}° = {uv.phase_center_ra:.6f} rad")
    print(f"  Dec: {np.degrees(uv.phase_center_dec):.6f}° = {uv.phase_center_dec:.6f} rad")
    
    print("\n📊 DATA ARRAY")
    print("-" * 70)
    print(f"  Shape:            {uv.data_array.shape}")
    print(f"  Format:           (Nblts={uv.Nblts}, Nspws=1, Nfreqs={uv.Nfreqs}, Npols={uv.Npols})")
    print(f"  Data type:        {uv.data_array.dtype}")
    print(f"  Memory size:      {uv.data_array.nbytes / 1e6:.1f} MB")
    
    print("\n📈 DATA STATISTICS")
    print("-" * 70)
    data = uv.data_array[~uv.flag_array]  # Only unflagged data
    if len(data) > 0:
        print(f"  Real min/max:     {data.real.min():.3e} / {data.real.max():.3e}")
        print(f"  Imag min/max:     {data.imag.min():.3e} / {data.imag.max():.3e}")
        print(f"  Amplitude min/max: {np.abs(data).min():.3e} / {np.abs(data).max():.3e}")
        print(f"  Mean amplitude:   {np.abs(data).mean():.3e}")
        print(f"  Std amplitude:    {np.abs(data).std():.3e}")
    
    print("\n🚩 FLAG STATISTICS")
    print("-" * 70)
    flag_percent = 100 * uv.flag_array.sum() / uv.flag_array.size
    print(f"  Flagged data:     {flag_percent:.2f}%")
    print(f"  Unflagged data:   {100 - flag_percent:.2f}%")
    
    print("\n" + "="*70)

# Display metadata (uncomment after loading file)
# display_metadata(uv)

# =============================================================================
# SECTION 3: Interactive Visualization Functions
# =============================================================================

def plot_visibility_overview(uv, baseline_idx=0, freq_idx=0, pol_idx=0):
    """Plot visibility amplitude and phase for a specific baseline"""
    
    # Get unique baselines
    unique_bls = np.unique(uv.baseline_array)
    bl_num = unique_bls[baseline_idx]
    
    # Get antenna numbers
    ant1, ant2 = uv.baseline_to_antnums(bl_num)
    
    # Get data for this baseline
    bl_mask = uv.baseline_array == bl_num
    vis = uv.data_array[bl_mask, 0, freq_idx, pol_idx]
    flags = uv.flag_array[bl_mask, 0, freq_idx, pol_idx]
    times = uv.time_array[bl_mask]
    
    # Apply flags
    vis_unflagged = np.ma.masked_array(vis, mask=flags)
    
    # Create plot
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Amplitude
    axes[0].plot(times, np.abs(vis_unflagged), 'b.', markersize=2, alpha=0.6)
    axes[0].set_ylabel('Amplitude', fontsize=12)
    axes[0].set_title(f'Visibility: Baseline {ant1}-{ant2}, Freq {uv.freq_array[0,freq_idx]/1e6:.2f} MHz, Pol {pol_idx}', 
                      fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Phase
    axes[1].plot(times, np.angle(vis_unflagged, deg=True), 'g.', markersize=2, alpha=0.6)
    axes[1].set_ylabel('Phase (degrees)', fontsize=12)
    axes[1].set_ylim(-180, 180)
    axes[1].grid(True, alpha=0.3)
    
    # Real and Imaginary
    axes[2].plot(times, vis_unflagged.real, 'r.', markersize=2, alpha=0.4, label='Real')
    axes[2].plot(times, vis_unflagged.imag, 'b.', markersize=2, alpha=0.4, label='Imag')
    axes[2].set_ylabel('Real / Imag', fontsize=12)
    axes[2].set_xlabel('Time (JD)', fontsize=12)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n📊 Statistics for Baseline {ant1}-{ant2}:")
    print(f"  Number of time samples: {len(times)}")
    print(f"  Flagged samples: {flags.sum()} ({100*flags.sum()/len(flags):.1f}%)")
    if not flags.all():
        print(f"  Amplitude: {np.abs(vis_unflagged).mean():.3e} ± {np.abs(vis_unflagged).std():.3e}")

def plot_waterfall(uv, baseline_idx=0, pol_idx=0):
    """Plot waterfall (time-frequency) for a specific baseline"""
    
    # Get unique baselines
    unique_bls = np.unique(uv.baseline_array)
    bl_num = unique_bls[baseline_idx]
    ant1, ant2 = uv.baseline_to_antnums(bl_num)
    
    # Get data for this baseline
    bl_mask = uv.baseline_array == bl_num
    vis = uv.data_array[bl_mask, 0, :, pol_idx]  # All frequencies
    flags = uv.flag_array[bl_mask, 0, :, pol_idx]
    times = uv.time_array[bl_mask]
    
    # Apply flags
    vis_unflagged = np.ma.masked_array(np.abs(vis), mask=flags)
    
    # Create plot
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Amplitude waterfall
    im1 = axes[0].imshow(vis_unflagged.T, aspect='auto', cmap='viridis', 
                         origin='lower', interpolation='nearest')
    axes[0].set_xlabel('Time Index', fontsize=12)
    axes[0].set_ylabel('Frequency Channel', fontsize=12)
    axes[0].set_title(f'Amplitude Waterfall: Baseline {ant1}-{ant2}, Pol {pol_idx}', 
                      fontsize=14, fontweight='bold')
    plt.colorbar(im1, ax=axes[0], label='Amplitude')
    
    # Phase waterfall
    phase = np.ma.masked_array(np.angle(vis, deg=True), mask=flags)
    im2 = axes[1].imshow(phase.T, aspect='auto', cmap='twilight', 
                         origin='lower', interpolation='nearest', vmin=-180, vmax=180)
    axes[1].set_xlabel('Time Index', fontsize=12)
    axes[1].set_ylabel('Frequency Channel', fontsize=12)
    axes[1].set_title('Phase Waterfall', fontsize=14, fontweight='bold')
    plt.colorbar(im2, ax=axes[1], label='Phase (degrees)')
    
    plt.tight_layout()
    plt.show()

def plot_spectrum(uv, baseline_idx=0, time_idx=0, pol_idx=0):
    """Plot spectrum (amplitude vs frequency) for a specific baseline and time"""
    
    # Get unique baselines
    unique_bls = np.unique(uv.baseline_array)
    bl_num = unique_bls[baseline_idx]
    ant1, ant2 = uv.baseline_to_antnums(bl_num)
    
    # Get data for this baseline
    bl_mask = uv.baseline_array == bl_num
    bl_indices = np.where(bl_mask)[0]
    
    if time_idx >= len(bl_indices):
        print(f"Time index {time_idx} out of range (max: {len(bl_indices)-1})")
        return
    
    idx = bl_indices[time_idx]
    
    vis = uv.data_array[idx, 0, :, pol_idx]
    flags = uv.flag_array[idx, 0, :, pol_idx]
    freqs = uv.freq_array[0, :] / 1e6  # Convert to MHz
    
    # Apply flags
    vis_unflagged = np.ma.masked_array(vis, mask=flags)
    
    # Create plot
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Amplitude spectrum
    axes[0].plot(freqs, np.abs(vis_unflagged), 'b-', linewidth=1)
    axes[0].set_ylabel('Amplitude', fontsize=12)
    axes[0].set_title(f'Spectrum: Baseline {ant1}-{ant2}, Time {time_idx}, Pol {pol_idx}', 
                      fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Phase spectrum
    axes[1].plot(freqs, np.angle(vis_unflagged, deg=True), 'g-', linewidth=1)
    axes[1].set_ylabel('Phase (degrees)', fontsize=12)
    axes[1].set_xlabel('Frequency (MHz)', fontsize=12)
    axes[1].set_ylim(-180, 180)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_uv_coverage(uv):
    """Plot UV coverage (baseline distribution)"""
    
    # Get UV coordinates
    u = uv.uvw_array[:, 0]
    v = uv.uvw_array[:, 1]
    
    # Plot
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    ax.plot(u, v, 'b.', markersize=1, alpha=0.3, label='UV points')
    ax.plot(-u, -v, 'b.', markersize=1, alpha=0.3)  # Conjugate points
    
    ax.set_xlabel('U (wavelengths)', fontsize=12)
    ax.set_ylabel('V (wavelengths)', fontsize=12)
    ax.set_title('UV Coverage', fontsize=14, fontweight='bold')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 UV Coverage Statistics:")
    print(f"  U range: {u.min():.1f} to {u.max():.1f} wavelengths")
    print(f"  V range: {v.min():.1f} to {v.max():.1f} wavelengths")
    print(f"  Max baseline: {np.sqrt(u**2 + v**2).max():.1f} wavelengths")

# =============================================================================
# SECTION 4: Interactive Widgets (Optional)
# =============================================================================

try:
    from ipywidgets import interact, IntSlider, Dropdown
    
    def create_interactive_plots(uv):
        """Create interactive plots with widgets"""
        
        print("🎮 Interactive Controls:")
        print("   Use the sliders below to explore different baselines, times, and frequencies\n")
        
        # Baseline selector
        @interact(
            baseline=IntSlider(min=0, max=uv.Nbls-1, step=1, value=0, description='Baseline:'),
            freq=IntSlider(min=0, max=uv.Nfreqs-1, step=1, value=0, description='Frequency:'),
            pol=IntSlider(min=0, max=uv.Npols-1, step=1, value=0, description='Polarization:')
        )
        def update_visibility_plot(baseline, freq, pol):
            plot_visibility_overview(uv, baseline, freq, pol)
        
        return update_visibility_plot
    
    WIDGETS_AVAILABLE = True
    
except ImportError:
    print("💡 Tip: Install ipywidgets for interactive controls: pip install ipywidgets")
    WIDGETS_AVAILABLE = False

# =============================================================================
# SECTION 5: Usage Examples
# =============================================================================

print("""
📚 USAGE EXAMPLES
================

After loading your UVFITS file into variable 'uv', try these commands:

1. Display metadata:
   display_metadata(uv)

2. Plot visibility for first baseline:
   plot_visibility_overview(uv, baseline_idx=0, freq_idx=0, pol_idx=0)

3. Plot waterfall diagram:
   plot_waterfall(uv, baseline_idx=0, pol_idx=0)

4. Plot spectrum:
   plot_spectrum(uv, baseline_idx=0, time_idx=0, pol_idx=0)

5. Plot UV coverage:
   plot_uv_coverage(uv)

6. Interactive exploration (if widgets available):
   create_interactive_plots(uv)

7. Access raw data:
   visibilities = uv.data_array
   times = uv.time_array
   freqs = uv.freq_array
   baselines = uv.baseline_array

8. Get specific visibility:
   bl_idx = 0
   vis = uv.data_array[bl_idx, 0, :, :]  # All freqs and pols for one baseline-time
""")

✅ Imports successful!
   pyuvdata available: True
   astropy available: True

💡 Tip: Install ipywidgets for file browser: pip install ipywidgets
💡 Tip: Install ipywidgets for interactive controls: pip install ipywidgets

📚 USAGE EXAMPLES

After loading your UVFITS file into variable 'uv', try these commands:

1. Display metadata:
   display_metadata(uv)

2. Plot visibility for first baseline:
   plot_visibility_overview(uv, baseline_idx=0, freq_idx=0, pol_idx=0)

3. Plot waterfall diagram:
   plot_waterfall(uv, baseline_idx=0, pol_idx=0)

4. Plot spectrum:
   plot_spectrum(uv, baseline_idx=0, time_idx=0, pol_idx=0)

5. Plot UV coverage:
   plot_uv_coverage(uv)

6. Interactive exploration (if widgets available):
   create_interactive_plots(uv)

7. Access raw data:
   visibilities = uv.data_array
   times = uv.time_array
   freqs = uv.freq_array
   baselines = uv.baseline_array

8. Get specific visibility:
   bl_idx = 0
   vis = uv.data_array[bl_idx, 0, :, :]  # All freqs and pols for on

In [10]:
uv = load_uvfits("/home/jay/projects/cuda-spatial-filtering/build/apps/hdf5_trial_vis.uvfits")

📖 Loading: /home/jay/projects/cuda-spatial-filtering/build/apps/hdf5_trial_vis.uvfits


AttributeError: 'NoneType' object has no attribute 'parnames'

In [17]:
with fits.open('/home/jay/projects/cuda-spatial-filtering/build/apps/hdf5_trial_vis.uvfits') as hdul:
        # 2. Print a summary of the contents
        print("HDU List Summary:")
        hdul.info()
        
        # 3. Access the data and header of a specific HDU
        # Primary HDU is usually index 0
        primary_data = hdul[1].data
        primary_header = hdul[0].header

HDU List Summary:
Filename: /home/jay/projects/cuda-spatial-filtering/build/apps/hdf5_trial_vis.uvfits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      14   ()      
  1  UV_DATA       1 BinTableHDU     33   528R x 9C   [1D, 1D, 1D, 1D, 1D, 1J, 32C, 1J, 1J]   


In [22]:
primary_data[0]

(np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.999994212962963), np.int32(257), array([1.706000e+03   +0.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j, 4.938539e+07   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 7.100000e+02-1970.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       7.100000e+02+1970.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j, 0.000000e+00   +0.j,
       0.000000e+00   +0.j, 0.000000e+00   +0.j], dtype='>c8'), np.int32(0), np.int32(-1))